In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_3.csv")

In [3]:
df

,timestamp_local,temp,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,430,1782.5,60.7,2.7,533.7,377.00,253.3
...,...,...,...,...,...,...,...,...,...
20973,23/06/2024 19:00,39.3,92,155.6,21.0,105.3,57.0,31.33,17.0
20974,23/06/2024 20:00,37.0,98,191.4,30.0,80.0,58.0,34.00,20.0
20975,23/06/2024 21:00,35.6,98,233.6,39.7,60.7,58.0,34.33,23.7
20976,23/06/2024 22:00,34.4,99,275.8,49.3,41.3,58.0,34.67,27.3


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data = df[['pm10']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled = scaled.reshape((20978,))

In [8]:
scaled.shape

(20978,)

In [9]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [10]:
def train_test_split(X, y, test_size=0.2):
    split_idx = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    return X_train, X_test, y_train, y_test

In [12]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

X_max = X_train.max()
X_min = X_train.min()

In [13]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [14]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0090 - root_mean_squared_error: 0.0932 - val_loss: 0.0030 - val_root_mean_squared_error: 0.0541 - learning_rate: 0.0010
Epoch 2/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0059 - root_mean_squared_error: 0.0770 - val_loss: 0.0029 - val_root_mean_squared_error: 0.0532 - learning_rate: 0.0010
Epoch 3/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0058 - root_mean_squared_error: 0.0760 - val_loss: 0.0030 - val_root_mean_squared_error: 0.0538 - learning_rate: 0.0010
Epoch 4/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0056 - root_mean_squared_error: 0.0750 - val_loss: 0.0029 - val_root_mean_squared_error: 0.0530 - learning_rate: 0.0010
Epoch 5/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0058 - root_mean_squared_error: 0.0759 - val_loss: 0.0028 - val_root_mean_squared_error: 0.0522 - learning_rate: 0.0010
Epoch 6/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0055 - root_mean_squared_err

In [15]:
forecast = model.predict(X_test)

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [16]:
poll = np.array(df["pm10"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [17]:
from sklearn.metrics import mean_squared_error

In [18]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 4.7310667


In [19]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 2.789744921028614


In [20]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 97.21025507897139 %


In [21]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 3.437428


In [22]:
y_test_true.shape

(4129, 168)

In [23]:
testPredict.shape

(4129, 168)

In [24]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [25]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 0.9601
RMSE for segment 2: 1.4815
RMSE for segment 3: 1.9684
RMSE for segment 4: 2.3427
RMSE for segment 5: 2.6567
RMSE for segment 6: 2.8961
RMSE for segment 7: 3.0552
RMSE for segment 8: 3.1668
RMSE for segment 9: 3.2467
RMSE for segment 10: 3.3053
RMSE for segment 11: 3.3546
RMSE for segment 12: 3.4010
RMSE for segment 13: 3.4382
RMSE for segment 14: 3.4615
RMSE for segment 15: 3.4873
RMSE for segment 16: 3.5026
RMSE for segment 17: 3.5183
RMSE for segment 18: 3.5280
RMSE for segment 19: 3.5387
RMSE for segment 20: 3.5497
RMSE for segment 21: 3.5550
RMSE for segment 22: 3.5709
RMSE for segment 23: 3.5873
RMSE for segment 24: 3.6142
RMSE for segment 25: 3.6611
RMSE for segment 26: 3.7248
RMSE for segment 27: 3.7931
RMSE for segment 28: 3.8674
RMSE for segment 29: 3.9429
RMSE for segment 30: 4.0083
RMSE for segment 31: 4.0663
RMSE for segment 32: 4.1144
RMSE for segment 33: 4.1492
RMSE for segment 34: 4.1909
RMSE for segment 35: 4.2248
RMSE for segment 36: 4.2511
R

In [26]:
forecast

array([[0.2030347 , 0.19288543, 0.18089303, ..., 0.2614551 , 0.26943007,
        0.27735844],
       [0.18327582, 0.1701877 , 0.16420816, ..., 0.27413404, 0.28114527,
        0.28355607],
       [0.17302224, 0.16585366, 0.16838379, ..., 0.28111798, 0.28522503,
        0.278835  ],
       ...,
       [0.04296872, 0.04470668, 0.04830423, ..., 0.05445162, 0.05342598,
        0.05364693],
       [0.04467445, 0.04587761, 0.04731631, ..., 0.0536266 , 0.05364816,
        0.0559947 ],
       [0.04478212, 0.04442422, 0.04534929, ..., 0.05328661, 0.05514623,
        0.0591929 ]], dtype=float32)

In [27]:
testPredict

array([[129.69357 , 128.81975 , 127.78725 , ..., 134.72336 , 135.40997 ,
        136.09259 ],
       [127.99239 , 126.865555, 126.35074 , ..., 135.81497 , 136.41862 ,
        136.62617 ],
       [127.1096  , 126.49241 , 126.71024 , ..., 136.41626 , 136.76987 ,
        136.21971 ],
       ...,
       [115.91244 , 116.06207 , 116.37181 , ..., 116.90108 , 116.812775,
        116.831795],
       [116.059296, 116.16289 , 116.28675 , ..., 116.83005 , 116.8319  ,
        117.033936],
       [116.068565, 116.03775 , 116.1174  , ..., 116.80077 , 116.960884,
        117.30929 ]], dtype=float32)

In [28]:
y_test_true

array([[129.11922, 128.78108, 128.45549, ..., 141.64235, 142.18085,
        142.73186],
       [128.78108, 128.45549, 128.11736, ..., 142.18085, 142.73186,
        143.27036],
       [128.45549, 128.11736, 129.7078 , ..., 142.73186, 143.27036,
        142.18085],
       ...,
       [115.59423, 115.71946, 115.84469, ..., 118.34932, 118.47455,
        118.59978],
       [115.71946, 115.84469, 116.13272, ..., 118.47455, 118.59978,
        118.59978],
       [115.84469, 116.13272, 116.43328, ..., 118.59978, 118.59978,
        118.59978]], dtype=float32)

In [29]:
model.save("pm10_1dcnn.keras")